In [ ]:
import pdfplumber
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

# Инициализируем модель и токенизатор
model = AutoModelForCausalLM.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Устанавливаем устройство (CPU или GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def clean_text(text):
    text = re.sub('[^a-zA-Z0-9а-яА-ЯёЁ .,:;!?#$%&*()_+|~{}[];<>@"\'\\\\]', ' ', text)
    text = re.sub(' +', ' ', text).strip()
    return text

def tokenize_in_chunks(tokens, chunk_size=3):
    return [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.texts[idx], dtype=torch.long)

def collate_fn(batch):
    # Выравнивание последовательностей с использованием padding
    max_length = max(len(x) for x in batch)
    padded_batch = [torch.cat([x, torch.zeros(max_length - len(x), dtype=torch.long)]) for x in batch]
    return torch.stack(padded_batch)

# Чтение PDF-документа
with pdfplumber.open('p2.pdf') as pdf:
    pages = pdf.pages
    texts = []
    
    for page in pages:
        text = page.extract_text(x_tolerance=1, y_tolerance=1)

        if text:  # Проверяем, что текст не пустой
            cleaned_text = clean_text(text)
            sentences = cleaned_text.split('. ')  # Разделяем на предложения

            for sentence in sentences:
                tokens = tokenizer.encode(sentence, max_length=512, truncation=True)
                token_chunks = tokenize_in_chunks(tokens, chunk_size=3)
                texts.extend(token_chunks)

# Создаем набор данных и загрузчик данных с collate_fn
dataset = TextDataset(texts)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Оптимизатор
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Обучение модели
num_epochs = 3

model.train()
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()

        # Перемещаем данные на нужное устройство
        input_ids = batch.to(device)

        # Получаем выходы модели
        outputs = model(input_ids=input_ids, labels=input_ids)
        
        # Вычисляем потерю и выполняем обратное распространение ошибки
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        

# Генерация текста на основе обученной модели
# Генерация текста на основе обученной модели
model.eval()
prompt = "Введите начальное предложение"  # Замените на ваше начальное предложение
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

# Установка pad_token_id
model.config.pad_token_id = tokenizer.eos_token_id  # Установите pad_token_id

# Генерация текста
output = model.generate(input_ids, max_length=50, num_return_sequences=1, pad_token_id=model.config.pad_token_id)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)  # Печатаем сгенерированный текст